In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%%html
<style>
.cell-output-ipywidget-background {
    background-color: transparent !important;
}
:root {
    --jp-widgets-color: var(--vscode-editor-foreground);
    --jp-widgets-font-size: var(--vscode-editor-font-size);
}  
</style>

In [3]:
import asyncio
import httpx
import json
from lib.rl.episode import Episode, EpisodeCompletion
from lib.rl.ppo import PPOLoss
from lib.rl.recipe import ComponentConfig, TuneRecipeConfig
from lib.rl.trainer import ExploreOptions, Trainer, vLLMConfig
import random
import re
import torch
from torchtune.models.llama3_1 import llama3_1_8b
from typing import Any, AsyncIterable, Literal, Optional

with open("./data/chain-of-thought-examples.json") as f:
    chain_of_thought_examples: list[dict[str, str]] = json.load(f)

client = httpx.AsyncClient(
    timeout=httpx.Timeout(5.0, read=30.0),
    limits=httpx.Limits(max_connections=512, max_keepalive_connections=512),
)


async def sample_random_episode(
    difficulty: Literal["easy", "variable"] = "variable",
    example_probability: float = 0.0,
    max_prompt_characters: int = 8192,
    reward_follow_up_completion: bool = True,
    return_first_solver_as_winner: Optional[bool] = None,
    length_penalty: float = 0.0,
) -> Episode:
    while True:
        params: dict[str, Any] = {
            "difficulty": difficulty,
        }
        if return_first_solver_as_winner is not None:
            params["return_first_solver_as_winner"] = return_first_solver_as_winner
        try:
            response = await client.get(
                "http://0.0.0.0:2218/new-episode-data",
                params=params,
            )
            response.raise_for_status()
        except httpx.TimeoutException:
            continue
        result = response.json()
        prompt = result["prompt"]
        follow_up = result["follow_up"]
        solution = result["solution"]
        if len(prompt) <= max_prompt_characters:
            break

    async def reward_completion(completion: EpisodeCompletion) -> EpisodeCompletion:
        if len(completion.messages) == 2:
            follow_up_completion = await completion.follow_up(
                messages=[
                    {"role": "user", "content": follow_up},
                ],
                max_tokens=10
                + len("\n".join(f"{key}: {value}" for key, value in solution.items()))
                // 2,
            )
        else:
            follow_up_completion = completion
        answer = follow_up_completion.last_assistant_message.get("content")
        assert isinstance(answer, str)
        if reward_follow_up_completion:
            completion = follow_up_completion
        completion.reward = sum(
            [
                bool(
                    # Find first match of key followed by colon and capture following text
                    (
                        match := re.search(
                            rf"{key}: ([A-Za-z \.:-]+)",
                            answer,
                            re.IGNORECASE,
                        )
                    )
                    # Check if captured group matches expected value
                    and match.group(1).strip().lower() == value.strip().lower()
                )
                for key, value in solution.items()
            ]
        ) / len(solution)
        completion.reward -= (
            completion.all_absent_stop_tokens
            / (3 if reward_follow_up_completion else 2)
            / len(solution)
        )
        completion.reward -= (
            completion.completion_tokens
            / (len(prompt) + len(solution) * 10)
            * length_penalty
        )
        return completion

    async def on_sample(completions: list[EpisodeCompletion]) -> None:
        for completion in await asyncio.gather(
            *[reward_completion(completion) for completion in completions]
        ):
            completion.commit()

    example = random.choice(chain_of_thought_examples)

    return Episode(
        messages=[{"role": "user", "content": prompt}],
        examples=lambda: (
            [
                {"role": "user", "content": example["prompt"]},
                {
                    "role": "assistant",
                    "content": example["chain_of_thought"]
                    + (example["answer"] and f"\n\n---\n\n{example['answer']}"),
                },
            ]
            if random.random() < example_probability
            else []
        ),
        on_sample=on_sample,
    )


episodes_per_iteration = 64 * torch.cuda.device_count()


async def train_episodes() -> AsyncIterable[Episode | BaseException]:
    pending: set[asyncio.Task[Episode | BaseException]] = set()
    while True:
        pending.update(
            asyncio.create_task(sample_random_episode())
            for _ in range(episodes_per_iteration - len(pending))
        )
        done, pending = await asyncio.wait(pending, return_when=asyncio.FIRST_COMPLETED)
        for task in done:
            try:
                yield task.result()
            except BaseException as e:
                yield e


async def val_episodes() -> AsyncIterable[Episode | BaseException]:
    for fut in asyncio.as_completed(
        sample_random_episode() for _ in range(64 * torch.cuda.device_count())
    ):
        try:
            yield await fut
        except BaseException as e:
            yield e


model_name = "rl38"

trainer = Trainer(
    base_model="NousResearch/Hermes-2-Theta-Llama-3-8B",
    output_dir=f"./models/{model_name}",
    explore_options=ExploreOptions(
        iterations=7,
        num_parents=6,
        branch_factor=3,
        patience=5,
        sample_probability_power=None,
        sampling_kwargs={"max_tokens": 2048},
        # split_method="prob",
        # split_point_std_deviation=0.5,
    ),
    train_episodes=train_episodes(),
    episodes_per_iteration=episodes_per_iteration,
    max_mask_sequence_batch_size=1,
    val_episodes=val_episodes(),
    val_patience=15,
    val_samples_per_episode=3,
    val_sampling_kwargs={"max_tokens": 2048},
    tune_model=llama3_1_8b,
    tune_model_type="LLAMA3",
    tune_recipe_config=TuneRecipeConfig(
        seed=42,
        shuffle=True,
        num_output_chunks=4,
        resume_from_checkpoint=False,
        batch_size=1,
        epochs=1,
        # max_steps_per_epoch=32,
        optimizer=ComponentConfig(
            "torch.optim.AdamW",
            # "bitsandbytes.optim.PagedAdamW8bit",
            # "bitsandbytes.optim.AdamW",
            # params=PLACEHOLDER,
            lr=4e-6,
            fused=True,
        ),
        loss=ComponentConfig(
            PPOLoss,
            policy_coef=0.0,
            clip_epsilon=0.2,
            unclipped_policy_coef=0.0,
            tanh_log_policy_coef=0.8,
            value_coef=0.0,
            entropy_coef=0.0,
            entropy_target=0.6,
            entropy_target_coef=0.05,
            kl_coef=0.05,
            weighted_entropy_coef=0.2,
            weighted_kl_coef=0.0,
            weighted_ce_coef=0.0,
            normalize_values=False,
            normalize_advantages=False,
        ),
        compile=False,
        optimizer_in_bwd=False,
        gradient_accumulation_steps=1,
        enable_activation_checkpointing=True,
        enable_activation_offloading=False,
        custom_sharded_layers=["tok_embeddings", "output"],
        log_every_n_steps=1,
        log_peak_memory_stats=True,
    ),
    # tune_run=False,
    tune_sequence_length=16384,
    vllm_config=vLLMConfig(
        env={"VLLM_ALLOW_LONG_MAX_MODEL_LEN": "1"},
        kwargs=dict(
            block_size=32,
            disable_log_requests=True,
            enable_chunked_prefill=True,
            enable_prefix_caching=True,
            enforce_eager=True,
            gpu_memory_utilization=0.9,
            max_model_len=16384,
            max_num_seqs=512,
            max_num_batched_tokens=16384,
            preemption_mode="swap",
            return_tokens_as_token_ids=True,
            swap_space=100,
        ),
        max_concurrent_samples=512,
        min_time_between_requests=0.0,
        timeout=120 + 15 * torch.cuda.device_count(),
    ),
    wandb_kwargs=dict(
        name=model_name,
        id=model_name,
    ),
)

config.json:   0%|          | 0.00/716 [00:00<?, ?B/s]

INFO 12-17 15:22:24 llm_engine.py:237] Initializing an LLM engine (v0.6.3.post1) with config: model='NousResearch/Hermes-2-Theta-Llama-3-8B', speculative_config=None, tokenizer='NousResearch/Hermes-2-Theta-Llama-3-8B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=NousResearch/Hermes-2-Theta-Llama-3-8B, num_scheduler_steps=1, chunked_prefill_enabled=Fal

tokenizer_config.json:   0%|          | 0.00/56.3k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/169 [00:00<?, ?B/s]

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: bradhilton. Use `wandb login --relogin` to force relogin


In [4]:
def tokens(string: str) -> list[int]:
    tokens = trainer.tokenizer.llm.get_tokenizer().encode(
        string, add_special_tokens=False  # type: ignore
    )
    if len(tokens) > 1:
        return []
    return tokens


word_biases = [
    # Critical thinking markers (highest importance)
    ("wait", 0.9),  # Frequent correction initiator
    ("hmm", 0.8),  # Common uncertainty marker
    ("alternatively", 0.8),  # Key for exploring options
    # Important correction/uncertainty markers
    ("perhaps", 0.7),  # Common for tentative suggestions
    ("actually", 0.7),  # Important for corrections
    ("rather", 0.7),  # Refinement marker
    # Analysis progression markers
    ("first", 0.6),
    ("similarly", 0.6),
    ("therefore", 0.6),
    ("instead", 0.6),
    # Mistake acknowledgment
    ("wrong", 0.5),
    ("incorrect", 0.5),
    ("no", 0.5),
    # Supporting markers (lower importance)
    ("but", 0.4),
    ("so", 0.4),
    ("thus", 0.4),
    ("now", 0.4),
    ("given", 0.4),
    ("suppose", 0.4),
    # Light correction markers
    ("oh", 0.3),
    ("oops", 0.3),
    ("right", 0.3),
]

logit_bias = {
    token: bias / 2
    for word, bias in word_biases
    for transform in [lambda x: x, str.capitalize, lambda x: " " + x]
    for token in tokens(transform(word))
}
logit_bias

{11748: 0.45,
 14524: 0.45,
 3868: 0.45,
 81122: 0.4,
 88601: 0.4,
 93114: 0.4,
 69487: 0.4,
 66372: 0.35,
 32576: 0.35,
 8530: 0.35,
 74128: 0.35,
 53692: 0.35,
 3604: 0.35,
 74303: 0.35,
 65002: 0.35,
 4856: 0.35,
 3983: 0.3,
 5451: 0.3,
 1176: 0.3,
 68791: 0.3,
 30293: 0.3,
 55915: 0.3,
 9093: 0.3,
 65937: 0.3,
 31887: 0.3,
 4619: 0.3,
 35970: 0.25,
 30285: 0.25,
 5076: 0.25,
 63054: 0.25,
 41568: 0.25,
 15465: 0.25,
 2201: 0.25,
 2822: 0.25,
 912: 0.25,
 8248: 0.2,
 4071: 0.2,
 719: 0.2,
 708: 0.2,
 4516: 0.2,
 779: 0.2,
 34232: 0.2,
 45600: 0.2,
 8617: 0.2,
 3409: 0.2,
 7184: 0.2,
 1457: 0.2,
 43068: 0.2,
 22818: 0.2,
 2728: 0.2,
 23289: 0.2,
 2319: 0.15,
 12174: 0.15,
 14346: 0.15,
 65662: 0.15,
 56372: 0.15,
 1315: 0.15,
 6107: 0.15,
 1314: 0.15}

In [5]:
trainer.explore_options.sampling_kwargs["logit_bias"] = logit_bias

In [6]:
explore_result = await trainer.explore(verbosity=1)

Starting 1 vLLM servers...
$ vllm serve NousResearch/Hermes-2-Theta-Llama-3-8B --port=8000 --block-size=32 --disable-log-requests --enable-chunked-prefill --enable-prefix-caching --enforce-eager --gpu-memory-utilization=0.9 --max-model-len=16384 --max-num-seqs=512 --max-num-batched-tokens=16384 --preemption-mode=swap --return-tokens-as-token-ids --swap-space=100 --api-key=default
vLLM servers started succesfully. Logs can be found at ./logs/vllm.log


explore:   0%|          | 0/64 [00:00<?, ?episode/s]

In [ ]:
await trainer.tune(trainer.explore_results[-1])

In [10]:
await trainer.train(iterations=10, verbosity=1)

val:   0%|          | 0/64 [00:00<?, ?episode/s]

explore:   0%|          | 0/64 [00:00<?, ?episode/s]

Early stopping val evaluation due to expired patience (0 remaining episodes x 15 patience per episode = 0 seconds)
$ tune run lib.rl.recipe.TuneRecipe --config /home/ubuntu/atreides/experiments/models/rl35/config.yaml


1|19|Loss: 0.0137: 100%|██████████| 19/19 [05:51<00:00, 17.82s/it, entropy=0.4768, entropy_target=0.1232, kl_div=0.1285, policy=-0.0627, tanh_log_policy=0.0041, unclipped_policy=-0.1151, value=2.8588, weighted_ce=-0.1450, weighted_entropy=0.0109, weighted_kl_div=0.1807] 

Saved iteration 39 model files to /home/ubuntu/atreides/experiments/models/rl35/0039
Starting 1 vLLM servers...
$ vllm serve /home/ubuntu/atreides/experiments/models/rl35/0039 --port=8009 --block-size=32 --disable-log-requests --enable-chunked-prefill --enable-prefix-caching --enforce-eager --gpu-memory-utilization=0.9 --max-model-len=16384 --max-num-seqs=512 --max-num-batched-tokens=16384 --preemption-mode=swap --return-tokens-as-token-ids --swap-space=100 --api-key=default
vLLM servers started succesfully. Logs can be found at ./logs/vllm.log


...

Early stopping val evaluation due to expired patience (0 remaining episodes x 15 patience per episode = 0 seconds)
$ tune run lib.rl.recipe.TuneRecipe --config /home/ubuntu/atreides/experiments/models/rl35/config.yaml


1|16|Loss: 0.0051: 100%|██████████| 16/16 [04:57<00:00, 17.51s/it, entropy=0.4877, entropy_target=0.1123, kl_div=0.0803, policy=0.0129, tanh_log_policy=0.0012, unclipped_policy=0.0093, value=3.4596, weighted_ce=-0.0097, weighted_entropy=0.0276, weighted_kl_div=0.0043]  

Saved iteration 43 model files to /home/ubuntu/atreides/experiments/models/rl35/0043
Starting 1 vLLM servers...
$ vllm serve /home/ubuntu/atreides/experiments/models/rl35/0043 --port=8009 --block-size=32 --disable-log-requests --enable-chunked-prefill --enable-prefix-caching --enforce-eager --gpu-memory-utilization=0.9 --max-model-len=16384 --max-num-seqs=512 --max-num-batched-tokens=16384 --preemption-mode=swap --return-tokens-as-token-ids --swap-space=100 --api-key=default
vLLM servers started succesfully. Logs can be found at ./logs/vllm.log


val:   0%|          | 0/64 [00:00<?, ?episode/s]

Early stopping val evaluation due to expired patience (0 remaining episodes x 15 patience per episode = 0 seconds)
